# How to use runtime primitives on the QAL hub
Adapted from the VQE/VqD example

## Define the operator and circuit

In [1]:
# define Hamiltonian
from qiskit.opflow import PauliSumOp
H = PauliSumOp.from_list([('XYII', 1), ('IYZI', 2), ('IIZX', 3), ('XIII', 4), ('IYII', 5)])

print(H)

1.0 * XYII
+ 2.0 * IYZI
+ 3.0 * IIZX
+ 4.0 * XIII
+ 5.0 * IYII


In [2]:
from qiskit.circuit.library import EfficientSU2
qc = EfficientSU2(num_qubits=H.num_qubits, reps=1)

qc_with_meas = qc.measure_all(inplace=False)
qc_with_meas.decompose().draw(fold=120)

┌──────────┐┌──────────┐                                ┌──────────┐┌───────────┐ ░ ┌─┐         
   q_0: ┤ Ry(θ[0]) ├┤ Rz(θ[4]) ├────────────────────────■───────┤ Ry(θ[8]) ├┤ Rz(θ[12]) ├─░─┤M├─────────
        ├──────────┤├──────────┤                      ┌─┴─┐     ├──────────┤├───────────┤ ░ └╥┘┌─┐      
   q_1: ┤ Ry(θ[1]) ├┤ Rz(θ[5]) ├───────────■──────────┤ X ├─────┤ Ry(θ[9]) ├┤ Rz(θ[13]) ├─░──╫─┤M├──────
        ├──────────┤├──────────┤         ┌─┴─┐    ┌───┴───┴───┐┌┴──────────┤└───────────┘ ░  ║ └╥┘┌─┐   
   q_2: ┤ Ry(θ[2]) ├┤ Rz(θ[6]) ├──■──────┤ X ├────┤ Ry(θ[10]) ├┤ Rz(θ[14]) ├──────────────░──╫──╫─┤M├───
        ├──────────┤├──────────┤┌─┴─┐┌───┴───┴───┐├───────────┤└───────────┘              ░  ║  ║ └╥┘┌─┐
   q_3: ┤ Ry(θ[3]) ├┤ Rz(θ[7]) ├┤ X ├┤ Ry(θ[11]) ├┤ Rz(θ[15]) ├───────────────────────────░──╫──╫──╫─┤M├
        └──────────┘└──────────┘└───┘└───────────┘└───────────┘                           ░  ║  ║  ║ └╥┘
meas_0: ═════════════════════════════════════════════════════════════════════════════════════╩══╬══╬══╬═
                                                                                                ║  ║  ║ 
meas_1: ════════════════════════════════════════════════════════════════════════════════════════╩══╬══╬═
                                                                                                   ║  ║ 
meas_2: ═══════════════════════════════════════════════════════════════════════════════════════════╩══╬═
                                                                                                      ║ 
meas_3: ══════════════════════════════════════════════════════════════════════════════════════════════╩═

In [6]:
# instantiate optimizer
from qiskit.algorithms.optimizers import SPSA
optimizer = SPSA(maxiter=500)

## Define the primitives

In [3]:
from qiskit.primitives import Estimator, Sampler
import numpy as np
theta = np.random.rand(qc.num_parameters)

estimator = Estimator([qc], [H])
e_result = estimator.run([qc,qc], [H,H], [theta,theta])
print(e_result.result().values[0])


sampler = Sampler([qc_with_meas])
s_result = sampler.run([qc_with_meas], [theta])
print(s_result.result())

3.2893980883794938
SamplerResult(quasi_dists=[{0: 0.534463710126212, 1: 0.004759346688145246, 2: 0.018762429217203943, 3: 0.04655897498420882, 4: 0.013229186890750786, 5: 0.00893610673954754, 6: 0.13312645689634195, 7: 0.006630323958123508, 8: 0.06542138657166453, 9: 0.0003307544396084377, 10: 0.014691590522237775, 11: 0.007511114365078926, 12: 0.07431594073231416, 13: 0.002837905609686144, 14: 0.0512055181112582, 15: 0.017219254147617474}], metadata=[{}])


## Build a VQE from Qiskit interfaces


In [4]:
from qiskit.algorithms import MinimumEigensolver, VQEResult
class CustomVQE(MinimumEigensolver):
    
    def __init__(self, estimator, circuit, optimizer, callback=None):
        self._estimator = estimator
        self._circuit = circuit
        self._optimizer = optimizer
        self._callback = callback
        
    def compute_minimum_eigenvalue(self, operator, aux_operators=None):
                
        # define objective
        def objective(x):
            e_job = self._estimator.run([self._circuit], [operator], [x])
            value = e_job.result().values[0]
            if self._callback:
                self._callback(value)
            return value
            
        # run optimization
        x0 = np.random.rand(self._circuit.num_parameters)
        res = self._optimizer.minimize(objective, x0=x0)
        
        # populate results
        result = VQEResult()
        result.cost_function_evals = res.nfev
        result.eigenvalue = res.fun
        result.optimal_parameters = res.x
        return result

## Run Locally

In [7]:
# run the custom VQE function
custom_vqe = CustomVQE(estimator, qc, optimizer)
result = custom_vqe.compute_minimum_eigenvalue(H)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 1000,
    'eigenstate': None,
    'eigenvalue': -12.999144071543924,
    'optimal_circuit': None,
    'optimal_parameters': array([ 1.73279166e+00,  1.59667517e+00,  1.56966106e+00,  1.57014006e+00,
        1.14572262e+00, -7.79229503e-03,  3.14293253e+00,  6.37334775e-04,
        3.68947714e-01,  1.56746095e+00, -4.05283146e-03, -1.04721141e-03,
       -1.12005098e+00,  1.07232319e+00,  1.57437810e+00,  3.13893708e+00]),
    'optimal_point': None,
    'optimal_value': None,
    'optimizer_evals': None,
    'optimizer_result': None,
    'optimizer_time': None}


## Run on IBMQ through runtime Session

In [13]:
from qiskit_ibm_runtime import (Session, Options, QiskitRuntimeService,
                                Sampler as RuntimeSampler, 
                                Estimator as RuntimeEstimator)


ibmq_token = ""
hub = "ibm-q-qal"
group = "escience"  # examnple 'escience'
project = "qradio"  # example qradio                           

service = QiskitRuntimeService(channel="ibm_quantum", token=ibmq_token, instance=hub + "/" + group + "/" + project)

backend = service.backend('ibmq_belem')

In [14]:

with Session(service=service, backend=backend) as session:

    # prepare primitives
    rt_estimator = RuntimeEstimator(session=session)
    
    # set up algorithm    
    rt_vqd = CustomVQE(rt_estimator, qc, optimizer)
    
    # run algorithm
    result = rt_vqd.compute_minimum_eigenvalue(H)
    print(result)    

RuntimeJobFailureError: 'Unable to retrieve job result. Job cdbc0asol6riq8djs520 has failed:\n2022-10-24T16:41:43.245356673Z /opt/app-root/lib64/python3.8/site-packages/qiskit_ibm_runtime/qpy/interface.py:255: UserWarning: The qiskit version used to generate the provided QPY file, 0.22.0, is newer than the current qiskit version 0.21.2. This may result in an error if the QPY file uses instructions not present in this current qiskit version\n2022-10-24T16:41:43.245356673Z   warnings.warn(\n2022-10-24T16:41:43.768578220Z --- Logging error ---\n2022-10-24T16:41:43.769605816Z Traceback (most recent call last):\n2022-10-24T16:41:43.769624919Z   File "/provider/programruntime/program_starter_wrapper.py", line 89, in execute\n2022-10-24T16:41:43.769624919Z     final_result = self.main(backend, self.messenger, **self.user_params)\n2022-10-24T16:41:43.769624919Z   File "/code/program.py", line 1164, in main\n2022-10-24T16:41:43.769624919Z     result = estimator.run(\n2022-10-24T16:41:43.7696341

In [ ]:
from qiskit.tools.jupyter import *

%qiskit_version_table
%qiskit_copyright